# Dashboard Dexscreener 

In [ ]:
#This script fetches live Solana-based token data from the Dex Screener API, 
#performs sentiment analysis, and ranks tokens based on trustworthiness & sentiment score.
#It outputs the top 15 positive sentiment tokens, saving detailed data to Google Sheets & Excel, while displaying a summary in the console.

#Key Filters: 
# --------- #
# Solana Blockchain Filter → Includes only tokens from Solana (chainId == "solana")
# Sentiment Analysis → Uses TextBlob to calculate sentiment scores from token descriptions.
# Liquidity & Price Data → Fetches USD liquidity & price for each token.
# Boosted Token Detection → Identifies if a token is recently boosted or has active boosts.
# Top 15 Tokens → Displays only the top 15 tokens with the highest positive sentiment.

#Workflow & Steps:
#---------------#
# Fetch Solana token profiles (name, description, links).
# Perform sentiment analysis on token descriptions.
# Retrieve liquidity & price data for each token.
# Check if the token is boosted or trending (Latest & Active Boosts).
# Calculate a Trustworthiness Score based on liquidity & boost status.
# Export full data to Google Sheets & Excel (Token Name, Sentiment, Liquidity, Price, Boosts).
# Display a summary table in the console (excluding long text fields like Description & Twitter).

In [3]:
import requests
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from textblob import TextBlob
from tabulate import tabulate

# 🔹 Google Sheets API Setup
SERVICE_ACCOUNT_FILE = "primal-index-449708-c1-5a0211dbb294.json"  # Replace with your JSON key file
GOOGLE_SHEET_NAME = "Dxscreener solana meme coin"  # Replace with your Google Sheet name

# 🔹 API Endpoints
TOKEN_PROFILES_URL = "https://api.dexscreener.com/token-profiles/latest/v1"
TOKEN_BOOSTS_URL = "https://api.dexscreener.com/token-boosts/latest/v1"
TOKEN_ACTIVE_BOOSTS_URL = "https://api.dexscreener.com/token-boosts/top/v1"
TOKEN_POOLS_URL = "https://api.dexscreener.com/token-pairs/v1/{chainId}/{tokenAddress}"

# 🔹 Function to Fetch API Data
def fetch_api_data(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return response.json()
        else:
            print(f"❌ Error: {response.status_code} from {url}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"❌ Request Error: {e}")
        return None

# 🔹 Function to Analyze Sentiment
def sentiment_score(text):
    if pd.isna(text) or not text:
        return 0  # Neutral if no description
    return TextBlob(str(text)).sentiment.polarity  # Sentiment score between -1 and +1

# 🔹 Function to Fetch Token Pool Data
def fetch_token_pools(chain_id, token_address):
    url = TOKEN_POOLS_URL.format(chainId=chain_id, tokenAddress=token_address)
    return fetch_api_data(url)

# 🔹 Function to Send Data to Google Sheets (Fixing NaN Issue)
def send_to_google_sheets(df):
    try:
        # Replace NaN values with an empty string
        pd.set_option('future.no_silent_downcasting', True)  # Explicitly opt-in to future behavior
        df = df.fillna("").infer_objects(copy=False)


        # Google Sheets Authentication
        scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
        creds = ServiceAccountCredentials.from_json_keyfile_name(SERVICE_ACCOUNT_FILE, scope)
        client = gspread.authorize(creds)
        
        # Open Google Sheet
        sheet = client.open(GOOGLE_SHEET_NAME).sheet1
        sheet.clear()  # Clear previous data
        
        # Convert DataFrame to List of Lists and Upload
        sheet.append_rows([df.columns.tolist()] + df.values.tolist())

        print(f"\n✅ Data successfully uploaded to Google Sheets: {GOOGLE_SHEET_NAME}")
    except Exception as e:
        print(f"❌ Google Sheets Upload Error: {e}")

# 🔹 Function to Consolidate Data
def consolidate_data():
    token_profiles = fetch_api_data(TOKEN_PROFILES_URL)
    if not token_profiles or not isinstance(token_profiles, list):
        print("⚠️ No valid token data fetched.")
        return

    df_tokens = pd.DataFrame(token_profiles)
    df_tokens = df_tokens[df_tokens["chainId"] == "solana"]  # Filter only Solana tokens
    df_tokens["Sentiment Score"] = df_tokens["description"].apply(sentiment_score)

    boosted_tokens = fetch_api_data(TOKEN_BOOSTS_URL)
    boosted_addresses = {t.get("tokenAddress", "") for t in boosted_tokens} if isinstance(boosted_tokens, list) else set()

    active_boosts = fetch_api_data(TOKEN_ACTIVE_BOOSTS_URL)
    active_boosted_addresses = {t.get("tokenAddress", "") for t in active_boosts} if isinstance(active_boosts, list) else set()

    results = []
    for _, row in df_tokens.iterrows():
        chain_id = row.get("chainId", "Unknown")
        token_address = row.get("tokenAddress", "Unknown")
        pool_data = fetch_token_pools(chain_id, token_address)

        base_token = pool_data[0].get("baseToken", {}).get("symbol", "Unknown") if pool_data and isinstance(pool_data, list) else "Unknown"
        liquidity = pool_data[0].get("liquidity", {}).get("usd", "N/A") if pool_data and isinstance(pool_data, list) else "N/A"
        price = pool_data[0].get("priceUsd", "N/A") if pool_data and isinstance(pool_data, list) else "N/A"

        trust_score = (float(liquidity) if liquidity != "N/A" else 0) + (1 if token_address in boosted_addresses else 0) + (1 if token_address in active_boosted_addresses else 0)

        results.append({
            "Base Token": base_token,
            "Token Name": row.get("header", "Unknown"),
            "Sentiment Score": row["Sentiment Score"],
            "Liquidity": liquidity,
            "Price (USD)": price,
            "Latest Boosted": "Yes" if token_address in boosted_addresses else "No",
            "Most Active Boosted": "Yes" if token_address in active_boosted_addresses else "No",
            "Trustworthiness Score": trust_score
        })

    df_final = pd.DataFrame(results)
    df_final = df_final.sort_values(by="Sentiment Score", ascending=False).head(15)  # Get top 15 positive tokens


    # Upload to Google Sheets
    send_to_google_sheets(df_final)

    # Display Summary Table in Console
    df_display = df_final.drop(columns=["Token Name", "Price (USD)"])
    print("\n📊 **Top 15 Solana Tokens by Sentiment Score** 📊\n")
    print(tabulate(df_display, headers='keys', tablefmt='fancy_grid', numalign="right", stralign="left"))

# 🔹 Run the function
consolidate_data()


❌ Google Sheets Upload Error: <Response [200]>

📊 **Top 15 Solana Tokens by Sentiment Score** 📊

╒════╤══════════════╤═══════════════════╤═════════════╤══════════════════╤═══════════════════════╤═════════════════════════╕
│    │ Base Token   │   Sentiment Score │   Liquidity │ Latest Boosted   │ Most Active Boosted   │   Trustworthiness Score │
╞════╪══════════════╪═══════════════════╪═════════════╪══════════════════╪═══════════════════════╪═════════════════════════╡
│  9 │ RSA          │          0.333333 │           0 │ No               │ No                    │                       0 │
├────┼──────────────┼───────────────────┼─────────────┼──────────────────┼───────────────────────┼─────────────────────────┤
│ 14 │ PYRAI        │              0.25 │     62430.5 │ No               │ No                    │                 62430.5 │
├────┼──────────────┼───────────────────┼─────────────┼──────────────────┼───────────────────────┼─────────────────────────┤
│ 12 │ GIGABOLZ     │       